# Tune_lgb
* 제출결과 : 

In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import log_loss
import lightgbm as lgb

from IPython.core.display import display, HTML
display(HTML('<style>.container {width:100% !important; }</style>'))

In [96]:
path = "./data/" 
train_df = pd.read_csv(path+'train.csv',index_col=0)
test_df = pd.read_csv(path+'test.csv',index_col=0)
sample_submission_df = pd.read_csv(path+'sample_submission.csv',index_col=0)
pd.options.display.max_columns = 30

## 전처리
 - 이상치 확인 및 처리

In [97]:
train_df.describe()

,fiberID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,fiberMag_i,fiberMag_z,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
count,199991.000000,1.999910e+05,199991.000000,199991.000000,199991.000000,199991.000000,1.999910e+05,199991.000000,199991.000000,199991.000000,199991.000000,199991.000000,199991.000000,199991.000000,199991.000000,199991.000000,199991.000000,199991.000000,199991.000000,199991.000000,199991.000000
mean,360.830152,-6.750146e+00,18.675373,18.401235,18.043495,17.663526,1.084986e+01,19.072693,19.134483,18.183331,18.000882,21.837903,18.454136,18.481525,17.686617,17.699207,20.110991,18.544375,18.181544,17.692395,17.189281
std,225.305890,1.187678e+04,155.423024,127.128078,116.622194,123.735298,4.172116e+03,749.256162,90.049058,122.378972,145.862346,789.472333,154.376277,97.240448,145.730872,142.691880,122.299062,161.728183,133.984475,131.183416,133.685138
min,1.000000,-5.310802e+06,-40022.466071,-27184.795793,-26566.310827,-24878.828280,-1.864766e+06,-215882.917191,-21802.656144,-20208.516262,-26505.602101,-24463.431833,-25958.752324,-23948.588523,-40438.184078,-30070.729379,-26236.578659,-36902.402336,-36439.638493,-38969.416822,-26050.710196
25%,174.000000,1.965259e+01,18.701180,18.048572,17.747663,17.425523,1.994040e+01,18.902851,18.259352,17.903615,17.606148,19.247795,18.113933,17.479794,17.050294,16.804705,19.266214,18.076120,17.423425,16.977671,16.705774
50%,349.000000,2.087136e+01,19.904235,19.454492,19.043895,18.611799,2.104910e+01,20.069038,19.631419,19.188763,18.710967,20.366848,19.586559,19.182789,18.693370,18.174592,20.406840,19.547674,19.143156,18.641756,18.100997
75%,526.000000,2.216043e+01,21.150297,20.515936,20.073528,19.883760,2.233754e+01,21.385830,20.773911,20.331419,20.133179,21.797480,21.004397,20.457491,20.019112,19.807652,21.992898,20.962386,20.408140,19.968846,19.819554
max,1000.000000,1.877392e+04,3538.984910,3048.110913,4835.218639,9823.740407,4.870154e+03,248077.513380,12084.735440,8059.638535,18358.921741,298771.019041,12139.815877,7003.136546,9772.190537,17403.789263,14488.251976,10582.058590,12237.951703,4062.499371,7420.534172


In [98]:
def del_outlier(data, min=0, max=60):
    up_idx_t=()
    dw_idx_t=()
    train_light = data.iloc[:,2:]
    for i in range(len(train_light.columns)):
        col = train_light.columns[i]
        up_idx_t+=tuple(data[data[col]>max].index)
        dw_idx_t+=tuple(data[data[col]<min].index)
    del_idx = set(up_idx_t+dw_idx_t)
    
    return data[~data.index.isin(del_idx)]

In [99]:
train_df = del_outlier(train_df, min=-60, max=60)

In [100]:
train_df.describe()

,fiberID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,fiberMag_i,fiberMag_z,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
count,199805.000000,199805.000000,199805.000000,199805.000000,199805.000000,199805.000000,199805.000000,199805.000000,199805.000000,199805.000000,199805.000000,199805.000000,199805.000000,199805.000000,199805.000000,199805.000000,199805.000000,199805.000000,199805.000000,199805.000000,199805.000000
mean,360.874257,20.984096,19.860971,19.286994,18.872327,18.612319,21.171538,20.089746,19.505260,19.087316,18.828446,20.710347,19.600430,19.005845,18.605146,18.391528,20.731087,19.530585,18.939735,18.520452,18.276576
std,225.326886,2.084378,1.868696,1.721250,1.628469,1.684458,1.982522,1.842191,1.706138,1.626217,1.676483,2.441525,2.098483,1.971022,1.906945,2.056474,2.187267,1.959114,1.859963,1.796639,1.864622
min,1.000000,-12.978725,-19.537079,-43.795556,-6.406284,-6.793444,-55.669722,-8.294703,-11.531777,10.646672,8.049850,-50.847000,-52.501232,-56.256807,-34.494076,-45.646493,3.444448,12.498143,10.286917,11.337467,10.977328
25%,174.000000,19.652802,18.701527,18.048615,17.747666,17.425886,19.941482,18.903532,18.259827,17.904548,17.607016,19.248446,18.114513,17.480249,17.051207,16.805175,19.265847,18.075890,17.423282,16.977685,16.705562
50%,349.000000,20.870785,19.903867,19.454103,19.043575,18.611026,21.049142,20.069213,19.632080,19.189421,18.711351,20.366762,19.586264,19.182726,18.693855,18.173881,20.405706,19.546802,19.142725,18.641701,18.100568
75%,526.000000,22.158644,21.148771,20.515044,20.072993,19.883188,22.336839,21.385460,20.773880,20.331692,20.133244,21.795629,21.002840,20.456383,20.018679,19.806769,21.990515,20.960706,20.407299,19.968434,19.819179
max,1000.000000,53.410871,54.250456,54.501228,49.883443,42.136572,54.055822,37.745406,32.574603,42.002047,36.498179,58.298437,58.042137,55.633809,59.835880,57.578250,49.487277,59.987842,33.666494,45.783550,48.276514


In [101]:
train_df.columns

Index(['type', 'fiberID', 'psfMag_u', 'psfMag_g', 'psfMag_r', 'psfMag_i',
       'psfMag_z', 'fiberMag_u', 'fiberMag_g', 'fiberMag_r', 'fiberMag_i',
       'fiberMag_z', 'petroMag_u', 'petroMag_g', 'petroMag_r', 'petroMag_i',
       'petroMag_z', 'modelMag_u', 'modelMag_g', 'modelMag_r', 'modelMag_i',
       'modelMag_z'],
      dtype='object')

## DATA Setting

In [102]:
column_number = {}
number_columns = {}
for i, column in enumerate(sample_submission_df.columns):
    column_number[column] = i
    number_columns[i] = column
    
    
def to_number(x, dic):
    return dic[x]

train_df['type_num'] = train_df['type'].apply(lambda x: to_number(x, column_number))


In [103]:
def add_minus_feature_all(data,test = False):
    '''
    combination으로 모든 변수들의 조합에 대해서 컬럼생성 
    각 mag에서 가장 높은값의 filter를 받아오는 "~~mag_MAX"생성
    결과 64개로 증가
    '''
    from itertools import combinations
    n = 0
    for count in range(5,21,5):
        s = 2
        if test == True :
            s = 1
        selected = data.columns[s:].values[n:count]
        mag = str.split(selected[0],'_')[0]
        data[mag+"_MAX"] = data[selected].apply(lambda x : str.split(np.argmax(x),'_')[1],axis = 1)
        for combi in list(combinations(selected,2)):
            name_1st = str.split(combi[0],'_')[1]
            name_2nd = str.split(combi[1],'_')[1]
            data[mag+"_"+name_1st+"-"+name_2nd] = data[combi[0]]-data[combi[1]]
        n=count
    return data

In [104]:
def add_minus_feature(data,test = False):
    '''
    근접변수에 대해서만 컬럼생성 ex) 'fiberMag_u-g', 'fiberMag_g-r', 'fiberMag_r-i', 'fiberMag_i-z'
    각 mag에서 가장 높은값의 filter를 받아오는 "~~mag_MAX"생성
    결과 38개로 증가 
    '''
    from itertools import combinations
    n = 0
    m = 0
    for count in range(5,21,5):
        s = 2
        if test == True :
            s = 1
        selected = data.columns[s:].values[n:count]
        mag = str.split(selected[0],'_')[0]
#         data[mag+"_MAX"] = data[selected].apply(lambda x : str.split(np.argmax(x),'_')[1],axis = 1)
        for i in range(0,len(selected)):        
            if i>0: 
                name_1st = str.split(m,'_')[1]
                name_2nd = str.split(selected[i],'_')[1]
                data[mag+"_"+name_1st+"-"+name_2nd] = data[m]-data[selected[i]]
            m = selected[i]    
        n=count
    return data

In [105]:
train_df = add_minus_feature(train_df)
test_df = add_minus_feature(test_df,True)

In [106]:
def label_encode(data):
    '''
    각 mag에서 가장 높은값의 filter를 labelencoding 
    '''
    from sklearn.preprocessing import LabelEncoder
    le = LabelEncoder()
    taget_cols = ['psfMag_MAX','fiberMag_MAX','petroMag_MAX','modelMag_MAX']
    for col in taget_cols:
        data[col] = le.fit_transform(data[col])
    return data

In [107]:
# train_df = label_encode(train_df)
# test_df = label_encode(test_df)

In [108]:
# train_df.drop(columns=['psfMag_u', 'psfMag_g', 'psfMag_r', 'psfMag_i',
#        'psfMag_z', 'fiberMag_u', 'fiberMag_g', 'fiberMag_r', 'fiberMag_i',
#        'fiberMag_z', 'petroMag_u', 'petroMag_g', 'petroMag_r', 'petroMag_i',
#        'petroMag_z', 'modelMag_u', 'modelMag_g', 'modelMag_r', 'modelMag_i',
#        'modelMag_z'], inplace=True)

# test_df.drop(columns=['psfMag_u', 'psfMag_g', 'psfMag_r', 'psfMag_i',
#        'psfMag_z', 'fiberMag_u', 'fiberMag_g', 'fiberMag_r', 'fiberMag_i',
#        'fiberMag_z', 'petroMag_u', 'petroMag_g', 'petroMag_r', 'petroMag_i',
#        'petroMag_z', 'modelMag_u', 'modelMag_g', 'modelMag_r', 'modelMag_i',
#        'modelMag_z'], inplace=True)


In [109]:
train_X = train_df.drop(columns=['type', 'type_num'], axis=1)
train_y = train_df['type_num']

In [110]:
train_X.columns

Index(['fiberID', 'psfMag_u', 'psfMag_g', 'psfMag_r', 'psfMag_i', 'psfMag_z',
       'fiberMag_u', 'fiberMag_g', 'fiberMag_r', 'fiberMag_i', 'fiberMag_z',
       'petroMag_u', 'petroMag_g', 'petroMag_r', 'petroMag_i', 'petroMag_z',
       'modelMag_u', 'modelMag_g', 'modelMag_r', 'modelMag_i', 'modelMag_z',
       'psfMag_u-g', 'psfMag_g-r', 'psfMag_r-i', 'psfMag_i-z', 'fiberMag_u-g',
       'fiberMag_g-r', 'fiberMag_r-i', 'fiberMag_i-z', 'petroMag_u-g',
       'petroMag_g-r', 'petroMag_r-i', 'petroMag_i-z', 'modelMag_u-g',
       'modelMag_g-r', 'modelMag_r-i', 'modelMag_i-z'],
      dtype='object')

In [111]:
# ## u-g, g-r, r-i, i-z 만 남기는게 좋지 않을까????
# train_X.drop(['psfMag_u-r','psfMag_u-i','psfMag_u-z','psfMag_g-i','psfMag_g-z','psfMag_r-z',
#              'fiberMag_u-r','fiberMag_u-i','fiberMag_u-z','fiberMag_g-i','fiberMag_g-z','fiberMag_r-z',
#              'petroMag_u-r','petroMag_u-i','petroMag_u-z','petroMag_g-i','petroMag_g-z','petroMag_r-z',
#              'modelMag_u-r','modelMag_u-i','modelMag_u-z','modelMag_g-i','modelMag_g-z','modelMag_r-z'] ,axis=1, inplace=True)

# test_X.drop(['psfMag_u-r','psfMag_u-i','psfMag_u-z','psfMag_g-i','psfMag_g-z','psfMag_r-z',
#              'fiberMag_u-r','fiberMag_u-i','fiberMag_u-z','fiberMag_g-i','fiberMag_g-z','fiberMag_r-z',
#              'petroMag_u-r','petroMag_u-i','petroMag_u-z','petroMag_g-i','petroMag_g-z','petroMag_r-z',
#              'modelMag_u-r','modelMag_u-i','modelMag_u-z','modelMag_g-i','modelMag_g-z','modelMag_r-z'] ,axis=1, inplace=True)

In [112]:
X_train, X_test, y_train, y_test = train_test_split(train_X, train_y, test_size=0.35, random_state=1059,stratify = train_y)

## Modeling

In [124]:
# fit a lightGBM model to the data
model = lgb.LGBMClassifier(
    learning_rate= 0.0032,
    max_depth= 5,
    num_leaves= 25,
    metric= ['multi_logloss', 'multi_logloss'],
    n_estimators= 10000,
    objective= 'multiclass',
    num_class= [train_y.nunique()],
    random_state= 1059,
    reg_alpha= 0.0,
    reg_lambda= 0.0,
    subsample_for_bin= 30000,
    bagging_fraction= 0.85,
    bagging_freq= 20,
    feature_fraction= 0.85,
    )

In [125]:
model.fit(X_train, y_train,
          sample_weight=None, init_score=None, eval_set=[(X_train, y_train),(X_test, y_test)], eval_names=['Train', 'Test'],
          eval_sample_weight=None, eval_class_weight=None, eval_init_score=None,
          eval_metric='logloss', early_stopping_rounds=100, verbose=100,
          feature_name='auto', callbacks=None
         )

Training until validation scores don't improve for 100 rounds
[100]	Train's multi_logloss: 1.52993	Test's multi_logloss: 1.53649
[200]	Train's multi_logloss: 1.17614	Test's multi_logloss: 1.18538
[300]	Train's multi_logloss: 0.953827	Test's multi_logloss: 0.964877
[400]	Train's multi_logloss: 0.800619	Test's multi_logloss: 0.813227
[500]	Train's multi_logloss: 0.691369	Test's multi_logloss: 0.705452
[600]	Train's multi_logloss: 0.611252	Test's multi_logloss: 0.626865
[700]	Train's multi_logloss: 0.5516	Test's multi_logloss: 0.568663
[800]	Train's multi_logloss: 0.50661	Test's multi_logloss: 0.525091
[900]	Train's multi_logloss: 0.472011	Test's multi_logloss: 0.491847
[1000]	Train's multi_logloss: 0.445129	Test's multi_logloss: 0.466305
[1100]	Train's multi_logloss: 0.423971	Test's multi_logloss: 0.446494
[1200]	Train's multi_logloss: 0.4067	Test's multi_logloss: 0.430524
[1300]	Train's multi_logloss: 0.392368	Test's multi_logloss: 0.417576
[1400]	Train's multi_logloss: 0.38066	Test's m

LGBMClassifier(bagging_fraction=0.85, bagging_freq=20, boosting_type='gbdt',
               class_weight=None, colsample_bytree=1.0, feature_fraction=0.85,
               importance_type='split', learning_rate=0.0032, max_depth=5,
               metric=['multi_logloss', 'multi_logloss'], min_child_samples=20,
               min_child_weight=0.001, min_split_gain=0.0, n_estimators=10000,
               n_jobs=-1, num_class=[19], num_leaves=25, objective='multiclass',
               random_state=1059, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=30000, subsample_freq=0)

In [126]:
# make predictions
train_y_pred = model.predict_proba(X_test)

In [127]:
real_y_pred = model.predict_proba(test_df)

In [128]:
#log_loss
log_loss(y_test,train_y_pred,labels= list(set(y_test)))

0.3398901934183096

In [129]:
feature_df = pd.DataFrame(model.feature_importances_, index= train_X.columns,columns= ['importance']).sort_values('importance',ascending = False)
feature_df.head(10)

,importance
fiberID,168983
psfMag_g-r,118813
modelMag_r-i,116537
fiberMag_r-i,116079
fiberMag_g-r,115075
modelMag_u-g,111338
modelMag_g-r,110600
psfMag_u-g,110471
fiberMag_u-g,109942
petroMag_u-g,108189


In [130]:
submission = pd.DataFrame(data=real_y_pred, columns=sample_submission_df.columns, index=sample_submission_df.index)
loss_v=log_loss(y_test,train_y_pred,labels= list(set(y_test)))
submission.to_csv(f'submission_lightbgm_cv_0229_4_({loss_v}).csv', index=True)

In [331]:
# #최적의 변수를 찾기 위한 KBest 함수 불러오기
# from sklearn.feature_selection import SelectKBest
# model_kbest = SelectKBest()
from sklearn.pipeline import Pipeline
lgbm_c = lgb.LGBMClassifier()
#파이프라인 구축
pipe_lgbm = Pipeline([('lgbm',lgbm_c)])
#파라미터 생성
params_lgbm = {
    "lgbm__max_depth": [9,10],
    "lgbm__learning_rate": [0.001, 0.003],
    "lgbm__num_leaves": [300,500],
    "lgbm__n_estimators": [500,1500],
    "lgbm__random_state": [1059],
    "lgbm__subsample_for_bin": [50000,200000],
    "lgbm__objective": ['multiclass'],
    "lgbm__num_class": [train_y.nunique()],
    "lgbm__metric": [['multi_logloss','multi_logloss']],
    "lgbm__reg_alpha": [0.0, 1.0],
    "lgbm__reg_lambda": [0.0, 1.0],
    "lgbm__feature_fraction": [0.85], 
    "lgbm__bagging_fraction": [0.85], 
    "lgbm__bagging_freq": [20],
}

In [332]:
#최적의 파라미터 탐색
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(pipe_lgbm, param_grid = params_lgbm,
                          cv=2, n_jobs=-1, verbose=10)

In [333]:
grid_search.fit(X_train, y_train)
lgbm_grid = grid_search.best_estimator_
lgbm_grid

Fitting 2 folds for each of 128 candidates, totalling 256 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed: 20.1min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed: 27.0min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed: 45.5min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed: 53.6min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 103.3min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 144.2min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed: 219.0min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed: 262.3min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed: 290.7min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed: 324.0min
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed: 411.9min
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed: 507.6min
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed: 566.5min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 601.2min
[Parallel(n_jobs=-1)]: Done 173 tasks      | e

Pipeline(memory=None,
         steps=[('lgbm',
                 LGBMClassifier(bagging_fraction=0.85, bagging_freq=20,
                                boosting_type='gbdt', class_weight=None,
                                colsample_bytree=1.0, feature_fraction=0.85,
                                importance_type='split', learning_rate=0.003,
                                max_depth=10,
                                metric=['multi_logloss', 'multi_logloss'],
                                min_child_samples=20, min_child_weight=0.001,
                                min_split_gain=0.0, n_estimators=1500,
                                n_jobs=-1, num_class=19, num_leaves=300,
                                objective='multiclass', random_state=1059,
                                reg_alpha=0.0, reg_lambda=0.0, silent=True,
                                subsample=1.0, subsample_for_bin=200000,
                                subsample_freq=0))],
         verbose=False)

In [334]:
print(grid_search.best_score_)

0.8783559448165321


In [341]:
grid_search.best_params_

{'lgbm__bagging_fraction': 0.85,
 'lgbm__bagging_freq': 20,
 'lgbm__feature_fraction': 0.85,
 'lgbm__learning_rate': 0.001,
 'lgbm__max_depth': 9,
 'lgbm__metric': ['multi_logloss', 'multi_logloss'],
 'lgbm__n_estimators': 500,
 'lgbm__num_class': 19,
 'lgbm__num_leaves': 300,
 'lgbm__objective': 'multiclass',
 'lgbm__random_state': 1059,
 'lgbm__reg_alpha': 0.0,
 'lgbm__reg_lambda': 0.0,
 'lgbm__subsample_for_bin': 50000}